In [1]:
import joblib
import os
import numpy as np
import pandas as pd
import datetime
import Funtions as fn
from sklearn.linear_model import LinearRegression

In [2]:
loaded_data = joblib.load('data/modal_biaya_harian_EMA.pkl')

In [3]:
loaded_data

,tgl_transaksi,total_modal_biaya
0,2008-01-01,1.629554e+08
1,2008-01-02,1.648955e+08
2,2008-01-03,1.633844e+08
3,2008-01-04,1.655167e+08
4,2008-01-05,1.665790e+08
...,...,...
360,2008-12-26,1.757288e+08
361,2008-12-27,1.759519e+08
362,2008-12-28,1.765858e+08
363,2008-12-29,1.781818e+08


In [4]:
PREDICTION_DAYS = 7
SEQUENCE_LENGTH = 30

In [5]:
scaler, data_scaled, X_train, y_train, X_val, y_val, train_dates, val_dates = fn.prepare_data(loaded_data, SEQUENCE_LENGTH)

In [6]:
joblib.dump(scaler, 'data/scaler.pkl')

['data/scaler.pkl']

In [7]:
dl_models = fn.create_ensemble_models((X_train.shape[1], 1))

In [8]:
trained_dl_models, histories = fn.train_ensemble_models(dl_models, X_train, y_train, X_val, y_val, 100)

Training model 1/3: lstm_model
Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - loss: 0.2557 - val_loss: 0.4085
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.1729 - val_loss: 0.2229
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0737 - val_loss: 0.0440
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0234 - val_loss: 0.0229
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0196 - val_loss: 0.0444
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0196 - val_loss: 0.0224
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0185 - val_loss: 0.0301
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0176 - val_loss: 0.0228
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0156 - val_loss: 0.0212
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0169 - val_loss: 0.0190
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0148 - val_loss: 0.0193
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22

In [9]:
individual_predictions_train, individual_predictions_val = fn.individual_predictions(trained_dl_models, X_train, X_val)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [10]:
train_preds = list(individual_predictions_train.values())
val_preds = list(individual_predictions_val.values())

X_stack_train = np.stack(train_preds, axis=1)
X_stack_val = np.stack(val_preds, axis=1)

In [11]:
# Model meta-ensemble
meta_model = LinearRegression()

meta_model.fit(X_stack_train, y_train)

LinearRegression()

In [12]:
# prediksi menggunakan meta model
ensemble_preds_hist_train = meta_model.predict(X_stack_train)
ensemble_preds_hist_val = meta_model.predict(X_stack_val)

In [13]:
last_date = loaded_data['tgl_transaksi'].iloc[-1]
if isinstance(last_date, pd.Timestamp):
    last_date = last_date.date()

future_dates = [last_date + datetime.timedelta(days=i+1) for i in range(PREDICTION_DAYS)]

last_sequence = data_scaled[-SEQUENCE_LENGTH:]

In [14]:
future_predictions = fn.predict_ensemble(trained_dl_models, scaler, last_sequence, meta_model)

In [15]:
# Menampilkan hasil prediksi masa depan dalam bentuk DataFrame
df_future_predictions = pd.DataFrame({
    'Tanggal': future_dates,
    'Prediksi Modal': future_predictions.flatten()
})

df_future_predictions.set_index('Tanggal', inplace=True)


In [16]:
df_future_predictions

,Prediksi Modal
Tanggal,
2008-12-31,1.787776e+08
2009-01-01,1.775262e+08
2009-01-02,1.770631e+08
2009-01-03,1.767604e+08
2009-01-04,1.764752e+08
2009-01-05,1.760586e+08
2009-01-06,1.757861e+08


In [17]:
fn.plot_ensemble_predictions(
    loaded_data,
    train_dates,
    val_dates,
    scaler,
    ensemble_preds_hist_train,
    ensemble_preds_hist_val,
    df_future_predictions
)

In [18]:
fn.plot_future_predictions(future_dates, future_predictions)

In [19]:
mape, mae, mse, rmse = fn.evaluate_model(y_val, ensemble_preds_hist_val)

Evaluasi hasil prediksi pada Data Validasi:
MAE  : 0.0638
MAPE : 9.7631%
MSE  : 0.0069
RMSE : 0.0833


In [22]:
save_dir = "model/peramalan_EMA"

for i, model in enumerate(trained_dl_models):
    model_name = model.name
    model_path = os.path.join(save_dir, f"{i+1}{model_name}.h5")
    model.save(model_path)
    print(f"Model {i+1} disimpan ke: {model_path}")


Model 1 disimpan ke: model/peramalan_EMA\1lstm_model.h5
Model 2 disimpan ke: model/peramalan_EMA\2gru_model.h5
Model 3 disimpan ke: model/peramalan_EMA\3bilstm_model.h5


In [23]:
joblib.dump(meta_model, 'model/peramalan_EMA/meta_model.pkl')

['model/peramalan_EMA/meta_model.pkl']